# Objective: The goal of this notebook is to teach an Agent to play tic-tac-toe. 

**Methodology**: We implement [Q-learning](https://en.wikipedia.org/wiki/Q-learning) from scratch with optional epsilon-greedy training, and ensemble modeling. The agent learns to play either first or second against a random player.

**Observations**: When teaching the agent to play in a single session, the agent learns fairly variant behaviors for starting a game as the first player and would suggest that 1) there are either several equally optimal first moves or 2) learning against a random player teaches the agent different behaviors, reinforcing the winning behaviors randomly. This led to implementing an ensemble approach to see if the agent could consistently find the most optimal first move. Several ensembles settled on a first move at the center of the board, which is known to be an acceptable first move according to known [strategy](https://en.wikipedia.org/wiki/Tic-tac-toe#Strategy). However, playing the opening move in a corner gives the opponent more chances to make a mistake. Even so either move does not make a difference between perfect players. I, personally, lost more often when the agent opened with a corner move but found it hard, if not impossible, to win when playing the opening move at all.

**Drawbacks**: The main drawback of this approach in general is that all states must be stored in memory. It's interesting that the number of unique states (when the agent must learn to play first or second) is 8,533. Given the seemingly small total action space at each step, it's easy to see how much one would need to save to memory to teach an agent a much more complex game.

**Next**: 
* Compare these results to that of deep Q-learning.
* Train two agents at the same time against each other.

# Imports

In [0]:
import numpy as np
import random
from tqdm import tqdm

# Define TicTacWhoa Class

In [0]:
class TicTacWhoa:
  def __init__(self, lr=.9, gamma=.9, epsilon=.2, episodes=100000):
   """Set learning rate, reward discount (gamma), probability of random move (epsilon),
   and number of episodes for training.
   """
   self.lr = lr
   self.gamma = gamma
   self.epsilon = epsilon
   self.episodes = episodes

  def _create_board(self):
    """Initialize clear board and check state space.
    """
    self.done = False
    self.board = np.array([3,4,5,6,7,8,9,10,11])
    self._check_or_add_state()

  def _check_or_add_state(self):
    """Check or add current board state in state dictionary. Set current action space.
    """
    self.state = str(list(self.board))
    if self.state_dict.get(self.state) is None:
      self.state_dict[self.state] = np.zeros(len(self.board[self.board > 2]))
    self.action_space = self.state_dict[self.state]
    if len(self.action_space) == 0:
      self.action_space = np.array([0])

  def _play_random(self, player):
    """Make a random play and check state space.
    """
    random_ind = random.choice(np.where(self.board > 2)[0])
    self.board[random_ind] = player
    self._check_or_add_state()
  
  def _choose_action_play_agent(self, possible_random=True):
    """Have agent choose an action.
    """
    if (possible_random) and (np.random.choice([1,2,3,4,5,6,7,8,9,10]) <= (self.epsilon*10)):
      self.action = np.argmax(np.random.rand(1,len(self.action_space)))
    else:
      if possible_random:
        self.action = np.argmax(self.action_space + np.random.randn(1,len(self.action_space))*(1./(self.i+1)))
      else:
        self.action = np.argmax(self.action_space)
    self.board[self.board[self.board > 2][self.action] - 3] = 1 
    self._check_or_add_state()

  def _is_finished(self):
    """Determine whether the game is over. Set reward.
    """
    self.reward = 0
    self.winner = 0

    if len(self.board[self.board>2]) == 0:
      self.winner = 3
      self.done = True

    shaped_board = self.board.reshape(3,3)
    for i in range(3):
      set_row = set(shaped_board[i])
      if len(set_row) == 1:
        self.winner = list(set_row)[0]
      set_col =  set(shaped_board[:,i])
      if len(set_col) == 1:
        self.winner = list(set_col)[0]

    set_left_diag = set(shaped_board.reshape(3,3).diagonal(0))
    if len(set_left_diag) == 1:
        self.winner = list(set_left_diag)[0]

    set_right_diag = set(np.flipud(shaped_board.reshape(3,3)).diagonal(0))
    if len(set_right_diag) == 1:
        self.winner = list(set_right_diag)[0]

    if self.winner == 2:
      self.reward = -1
      self.done = True
    elif self.winner == 1:
      self.reward = 1
      self.done = True

  def _step(self):
    """Move the game one step further. Determine if game is over.
    """
    self._is_finished()
    if self.winner == 0:
      self._play_random(2)
      self._is_finished()

  def _update_state_dict(self, current_state):
    """Update state dictionary with current information.
    """
    self.state_dict[current_state][self.action] = (1-self.lr)*self.state_dict[current_state][self.action] \
                                                  + self.lr*(self.reward \
                                                  + self.gamma*np.max(self.action_space))

  def learn(self, ensemble=False):
    """Teach agent to play tic-tac-toe. Agent will learn to go first and second.
    Train multiple agents with the ensemble parameter.
    """
    self.state_dict = {}
    self.rewards_lists = [np.zeros(self.episodes), np.zeros(self.episodes)]
    for i in range(self.episodes):
      self.i = i
      for j in range(2):
        self._create_board()
        if j == 1:
          self._step()
        for _ in range(9):
          current_state = self.state
          self._choose_action_play_agent()
          self._step()
          self._update_state_dict(current_state)
          if self.done == True:
              break
        self.rewards_lists[j][i] += self.reward
    if ensemble:
      return self.state_dict

  def learn_ensemble(self, repeats):
    """Teach multiple agents to play tic-tac-toe and combine them into one agent.
    """
    self.state_dict_ensemble = {}
    for i in tqdm(range(repeats)):
      state_dict = self.learn(ensemble=True)
      for state, actions in state_dict.items():
        if self.state_dict_ensemble.get(state) is not None:
          self.state_dict_ensemble[state] += actions
        else:
           self.state_dict_ensemble[state] = actions
    self.state_dict = self.state_dict_ensemble

  def _display_board(self):
    """Display current board.
    """
    new_board = []
    for b in self.board:
      if b == 1:
        new_board.append('X')
      elif b == 2:
        new_board.append('O')
      else:
        new_board.append(' ')
    print("""
     {} | {} | {} 
    ---|---|---
     {} | {} | {} 
    ---|---|---
     {} | {} | {} 
    """.format(new_board[0], new_board[1], new_board[2],
               new_board[3], new_board[4], new_board[5],
               new_board[6], new_board[7], new_board[8]))

  def _play_computer(self):
    """Have the agent make a move.
    """
    print('\nComputer made a move:')
    self._choose_action_play_agent(possible_random=False)

  def _play_human(self):
    """Allow human to make a move.
    """
    print('\nYou made a move:')
    ind = int(input('Select the index of your move: '))
    self.board[ind] = 2
    self._check_or_add_state()

  def play(self):
    """Play a game against the agent.
    """
    self._create_board()
    self._display_board()
    whos_first = input('Do you want to go first? (y/n): ')
    for i in range(9):
      if i % 2 == 0:
        if whos_first == 'y':
          self._play_human()
        else:
          self._play_computer()
      else:
        if whos_first == 'y':
          self._play_computer()
        else:
          self._play_human()
      self._display_board()
      self._is_finished()
      if self.done == True:
        print("Game Over!")
        break

# Teach Agent to play, and play against it!

In [0]:
ttw = TicTacWhoa(episodes=10000, epsilon=0)

In [4]:
ttw.learn_ensemble(repeats=100)

100%|██████████| 100/100 [13:23<00:00,  8.03s/it]


We let the agent play first, and it chooses the middle.

In [5]:
ttw.play()


       |   |   
    ---|---|---
       |   |   
    ---|---|---
       |   |   
    
Do you want to go first? (y/n): n

Computer made a move:

       |   |   
    ---|---|---
       | X |   
    ---|---|---
       |   |   
    

You made a move:
Select the index of your move: 0

     O |   |   
    ---|---|---
       | X |   
    ---|---|---
       |   |   
    

Computer made a move:

     O |   |   
    ---|---|---
       | X |   
    ---|---|---
     X |   |   
    

You made a move:
Select the index of your move: 2

     O |   | O 
    ---|---|---
       | X |   
    ---|---|---
     X |   |   
    

Computer made a move:

     O | X | O 
    ---|---|---
       | X |   
    ---|---|---
     X |   |   
    

You made a move:
Select the index of your move: 7

     O | X | O 
    ---|---|---
       | X |   
    ---|---|---
     X | O |   
    

Computer made a move:

     O | X | O 
    ---|---|---
       | X | X 
    ---|---|---
     X | O |   
    

You made a move:
Select the inde

Or we can play first and choose the corner. [The optimal move for the agent at this point is the center as well](https://en.wikipedia.org/wiki/Tic-tac-toe#Strategy).

In [6]:
ttw.play()


       |   |   
    ---|---|---
       |   |   
    ---|---|---
       |   |   
    
Do you want to go first? (y/n): y

You made a move:
Select the index of your move: 0

     O |   |   
    ---|---|---
       |   |   
    ---|---|---
       |   |   
    

Computer made a move:

     O |   |   
    ---|---|---
       | X |   
    ---|---|---
       |   |   
    

You made a move:
Select the index of your move: 8

     O |   |   
    ---|---|---
       | X |   
    ---|---|---
       |   | O 
    

Computer made a move:

     O |   |   
    ---|---|---
       | X |   
    ---|---|---
       | X | O 
    

You made a move:
Select the index of your move: 1

     O | O |   
    ---|---|---
       | X |   
    ---|---|---
       | X | O 
    

Computer made a move:

     O | O | X 
    ---|---|---
       | X |   
    ---|---|---
       | X | O 
    

You made a move:
Select the index of your move: 6

     O | O | X 
    ---|---|---
       | X |   
    ---|---|---
     O | X | O 
    

Comp